# EBNH (LHS of forces system)

Purpose: check if the operator `ENBH` (as implemented in PetIBM) is symmetric.

In [ ]:
import itertools
import numpy
from scipy.sparse import csr_matrix

import pyibm

from helper import *

In [ ]:
%matplotlib qt

In [ ]:
pyibm.__version__

In [ ]:
# Create the staggered Cartesian grids.
xlim, ylim = (-15.0, 15.0), (-15.0, 15.0)
xbox, ybox = (-0.75, 0.75), (-0.75, 0.75)
dx, dy = 0.01, 0.01  # grid spacings in the uniform region

config = {'x': {'start': xlim[0],
                'segments': [{'end': xbox[0],
                              'width': dx,
                              'stretching': 1.03,
                              'reverse': True},
                             {'end': xbox[1],
                              'width': dx,
                              'stretching': 1.0},
                             {'end': xlim[1],
                              'width': dx,
                              'stretching': 1.01}]},
          'y': {'start': ylim[0],
                'segments': [{'end': ybox[0],
                              'width': dy,
                              'stretching': 1.04,
                              'reverse': True},
                             {'end': ybox[1],
                              'width': dy,
                              'stretching': 1.0},
                             {'end': ylim[1],
                              'width': dy,
                              'stretching': 1.04}]}}

grid = pyibm.GridBase(config=config)  # vertices
gridc = pyibm.GridCellCentered(grid=grid)  # cell-centered
gridx = pyibm.GridFaceX(grid=grid)  # x-face centered
gridy = pyibm.GridFaceY(grid=grid)  # y-face centerred

# Display information about the grids.
print(gridc)
print(gridx)
print(gridy)

In [ ]:
# Create the Laplacian operator.
Re = 100.0  # Reynolds number
LHat = 1 / Re * pyibm.assemble_LHat(gridx, gridy)
print_matrix_info(LHat)

In [ ]:
# Create the approximate inverse of the operator A (velocity system).
dt = 0.01  # time-step size
BN = pyibm.assemble_BN(gridx, gridy, dt=dt, alpha=0.5, N=1, L=LHat)
print_matrix_info(BN)

In [ ]:
# Create the immersed boundary (circle).
radius = 0.5
xc, yc = 0.0, 0.0
ds = dx
N = int(round(2 * numpy.pi * radius / ds))
theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1]
x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
body = pyibm.Body(x, y, grid=gridc)
body

In [ ]:
# Create the scaling diagonal matrix MHat.
MHat = pyibm.assemble_MHat(gridx, gridy)
print_matrix_info(MHat)

In [ ]:
# Create the scaling diagonal matrix R.
R = pyibm.assemble_R(gridx, gridy)
print_matrix_info(R)

In [ ]:
# Set the delta kernel to use and support size.
delta_kernel = pyibm.delta_roma_et_al_1999
delta_kernel_size = 2

# Create the transfer kernel.
Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                          kernel=delta_kernel,
                          kernel_size=delta_kernel_size)
print_matrix_info(Op)

In [ ]:
# Create the interpolation operator (as implemented in PetIBM).
EHat = Op @ R @ MHat
print_matrix_info(EHat)

In [ ]:
# Create the speading operator (as implemented in PetIBM).
HHat = csr_matrix(Op.T)
print_matrix_info(HHat)

In [ ]:
# Create the LHS operator of the system for the Lagrangian forces.
EBNH = EHat @ BN @ HHat
print_matrix_info(EBNH)
# Check if operator is symmetric.
is_symmetric(EBNH)

In [ ]:
# Plot the non-zero structure of the operator.
plot_matrix(EBNH, cmap='viridis');

In [ ]:
condition_number(EBNH)

In [ ]:
Nmax = 3
xc_all = numpy.linspace(-dx, dx, num=11)
yc_all = numpy.linspace(-dx, dx, num=11)
ratios = [1.0, 1.5, 2.0, 2.5]

data = {}
for N in range(1, Nmax + 1):
    BN = pyibm.assemble_BN(gridx, gridy,
                           dt=dt, alpha=0.5, N=N, L=LHat)
    subdata = {}
    for r in ratios:
        xc_vals, yc_vals = [], []
        cond_vals = []
        for xc, yc in itertools.product(xc_all, yc_all):
            radius = 0.5
            ds = dx / r
            Nb = int(round(2 * numpy.pi * radius / ds))
            theta = numpy.linspace(0.0, 2 * numpy.pi, num=Nb + 1)[:-1]
            x = xc + radius * numpy.cos(theta)
            y = yc + radius * numpy.sin(theta)
            body = pyibm.Body(x, y, grid=gridc)
            Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                                      kernel=delta_kernel,
                                      kernel_size=delta_kernel_size)
            EHat = Op @ R @ MHat
            HHat = csr_matrix(Op.T)
            EBNH = EHat @ BN @ HHat
            xc_vals.append(xc)
            yc_vals.append(yc)
            cond_vals.append(condition_number(EBNH))
        subdata[f'r={r}'] = {'xc': xc_vals, 'yc': yc_vals,
                             'cond': cond_vals}
    data[f'N={N}'] = subdata

In [ ]:
fig, ax = pyplot.subplots(nrows=Nmax, ncols=len(ratios),
                          figsize=(10.0, 10.0))
for i, N in enumerate(range(1, Nmax + 1)):
    for j, r in enumerate(ratios):
        subdata = data[f'N={N}'][f'r={r}']
        cond = numpy.array([c[-1] for c in subdata['cond']])
        cond = numpy.absolute(cond)
        ax[i, j].set_title(f'N={N}, r={r}')
        ax[i, j].contourf(xc_all, yc_all, cond.reshape((xc_all.size,
                                                        yc_all.size)))
        ax[i, j].text(min(xc_all), min(yc_all),
                      'min={:.2e}\nmax={:.2e}'.format(min(cond), max(cond)))
        ax[i, j].axis('off')